# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/14_best_practices/14_demo_deployment_fastapi.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '14_demo_deployment_fastapi.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Démonstration : Déploiement ML avec FastAPI

Ce notebook illustre comment **déployer un modèle ML en production** avec FastAPI :

1. **API REST** : Endpoints pour prédictions
2. **Validation** : Pydantic pour valider les inputs
3. **Health Checks** : Endpoint `/health` pour monitoring
4. **Metrics** : Endpoint `/metrics` pour statistiques
5. **Dockerfile** : Conteneurisation
6. **Tests** : Requêtes HTTP avec `requests`

**Prérequis** : Modèle entraîné (voir notebook `11_demo_pipeline_complet.ipynb`)

In [ ]:
import numpy as np
import pandas as pd
import joblib
import json
from datetime import datetime
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, validator
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès !")

## 1. Préparation : Entraînement d'un Modèle Simple

Si le modèle n'existe pas déjà, nous en créons un rapidement.

In [ ]:
import os
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Vérifier si le modèle existe
model_dir = '/tmp/models'
model_path = os.path.join(model_dir, 'housing_api_model.joblib')

if os.path.exists(model_path):
    print(f"Modèle trouvé : {model_path}")
    model = joblib.load(model_path)
else:
    print("Modèle non trouvé. Entraînement d'un nouveau modèle...")
    
    # Chargement des données
    housing = fetch_california_housing()
    X, y = housing.data, housing.target  # type: ignore
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Pipeline simple
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1))
    ])
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Sauvegarde
    os.makedirs(model_dir, exist_ok=True)
    joblib.dump(model, model_path)
    print(f"Modèle entraîné et sauvegardé : {model_path}")

# Features du modèle
feature_names = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
print(f"\nFeatures attendues : {feature_names}")

## 2. Modèles Pydantic pour Validation des Inputs

In [ ]:
# Modèle Pydantic pour une seule prédiction
class HousingInput(BaseModel):
    """Schéma de validation pour une prédiction de prix de maison."""
    MedInc: float = Field(..., ge=0, le=15, description="Revenu médian (en 10k$)")
    HouseAge: float = Field(..., ge=1, le=52, description="Âge médian de la maison")
    AveRooms: float = Field(..., ge=1, le=20, description="Nombre moyen de pièces")
    AveBedrms: float = Field(..., ge=0.5, le=10, description="Nombre moyen de chambres")
    Population: float = Field(..., ge=1, le=10000, description="Population du quartier")
    AveOccup: float = Field(..., ge=1, le=20, description="Occupation moyenne")
    Latitude: float = Field(..., ge=32, le=42, description="Latitude")
    Longitude: float = Field(..., ge=-125, le=-114, description="Longitude")
    
    @validator('AveRooms')
    def validate_rooms(cls, v, values):
        """Vérifie que le nombre de pièces > chambres."""
        if 'AveBedrms' in values and v < values['AveBedrms']:
            raise ValueError('AveRooms doit être >= AveBedrms')
        return v
    
    class Config:
        schema_extra = {
            "example": {
                "MedInc": 3.5,
                "HouseAge": 20.0,
                "AveRooms": 5.0,
                "AveBedrms": 1.5,
                "Population": 1200.0,
                "AveOccup": 3.0,
                "Latitude": 37.5,
                "Longitude": -122.0
            }
        }

# Modèle pour batch de prédictions
class HousingBatchInput(BaseModel):
    """Schéma pour plusieurs prédictions."""
    data: List[HousingInput] = Field(..., min_items=1, max_items=100)

# Modèle de réponse
class PredictionResponse(BaseModel):
    """Schéma de réponse pour une prédiction."""
    prediction: float = Field(..., description="Prix prédit (en 100k$)")
    prediction_usd: str = Field(..., description="Prix prédit en dollars")
    model_version: str = Field(..., description="Version du modèle")
    timestamp: str = Field(..., description="Timestamp de la prédiction")

print("Modèles Pydantic définis !")
print("\nExemple d'input valide:")
print(json.dumps(HousingInput.Config.schema_extra['example'], indent=2))

## 3. Code de l'API FastAPI

Nous créons le fichier `api.py` contenant l'API complète.

In [ ]:
# Création du fichier api.py
api_code = '''
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field, validator
from typing import List, Dict
import joblib
import numpy as np
from datetime import datetime
import time

# Modèles Pydantic
class HousingInput(BaseModel):
    MedInc: float = Field(..., ge=0, le=15)
    HouseAge: float = Field(..., ge=1, le=52)
    AveRooms: float = Field(..., ge=1, le=20)
    AveBedrms: float = Field(..., ge=0.5, le=10)
    Population: float = Field(..., ge=1, le=10000)
    AveOccup: float = Field(..., ge=1, le=20)
    Latitude: float = Field(..., ge=32, le=42)
    Longitude: float = Field(..., ge=-125, le=-114)
    
    @validator('AveRooms')
    def validate_rooms(cls, v, values):
        if 'AveBedrms' in values and v < values['AveBedrms']:
            raise ValueError('AveRooms doit être >= AveBedrms')
        return v

class HousingBatchInput(BaseModel):
    data: List[HousingInput] = Field(..., min_items=1, max_items=100)

class PredictionResponse(BaseModel):
    prediction: float
    prediction_usd: str
    model_version: str
    timestamp: str

# Chargement du modèle
MODEL_PATH = "/tmp/models/housing_api_model.joblib"
model = joblib.load(MODEL_PATH)
MODEL_VERSION = "1.0.0"

# Métriques globales
metrics = {
    "total_predictions": 0,
    "total_requests": 0,
    "errors": 0,
    "start_time": datetime.now().isoformat()
}

# Application FastAPI
app = FastAPI(
    title="Housing Price Prediction API",
    description="API REST pour prédire les prix des maisons en Californie",
    version=MODEL_VERSION
)

@app.middleware("http")
async def add_metrics(request: Request, call_next):
    """Middleware pour tracker les requêtes."""
    metrics["total_requests"] += 1
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    response.headers["X-Process-Time"] = str(process_time)
    return response

@app.get("/")
def root():
    """Endpoint racine."""
    return {
        "message": "Housing Price Prediction API",
        "version": MODEL_VERSION,
        "endpoints": {
            "/predict": "POST - Prédiction unique",
            "/predict/batch": "POST - Prédictions batch",
            "/health": "GET - Health check",
            "/metrics": "GET - Métriques API",
            "/docs": "GET - Documentation Swagger"
        }
    }

@app.post("/predict", response_model=PredictionResponse)
def predict(input_data: HousingInput):
    """Prédiction unique."""
    try:
        # Conversion en array NumPy
        features = np.array([[
            input_data.MedInc,
            input_data.HouseAge,
            input_data.AveRooms,
            input_data.AveBedrms,
            input_data.Population,
            input_data.AveOccup,
            input_data.Latitude,
            input_data.Longitude
        ]])
        
        # Prédiction
        prediction = model.predict(features)[0]
        metrics["total_predictions"] += 1
        
        return PredictionResponse(
            prediction=float(prediction),
            prediction_usd=f"${prediction * 100000:.0f}",
            model_version=MODEL_VERSION,
            timestamp=datetime.now().isoformat()
        )
    except Exception as e:
        metrics["errors"] += 1
        raise HTTPException(status_code=500, detail=f"Erreur de prédiction: {str(e)}")

@app.post("/predict/batch")
def predict_batch(input_data: HousingBatchInput):
    """Prédictions batch."""
    try:
        # Conversion en array NumPy
        features_list = []
        for item in input_data.data:  # type: ignore
            features_list.append([
                item.MedInc, item.HouseAge, item.AveRooms, item.AveBedrms,
                item.Population, item.AveOccup, item.Latitude, item.Longitude
            ])
        features = np.array(features_list)
        
        # Prédictions
        predictions = model.predict(features)
        metrics["total_predictions"] += len(predictions)
        
        results = []
        for pred in predictions:
            results.append({
                "prediction": float(pred),
                "prediction_usd": f"${pred * 100000:.0f}"
            })
        
        return {
            "predictions": results,
            "count": len(results),
            "model_version": MODEL_VERSION,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        metrics["errors"] += 1
        raise HTTPException(status_code=500, detail=f"Erreur de prédiction: {str(e)}")

@app.get("/health")
def health_check():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "model_loaded": model is not None,
        "model_version": MODEL_VERSION,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/metrics")
def get_metrics():
    """Endpoint pour récupérer les métriques."""
    uptime_seconds = (datetime.now() - datetime.fromisoformat(metrics["start_time"])).total_seconds()
    return {
        **metrics,
        "uptime_seconds": uptime_seconds,
        "uptime_hours": uptime_seconds / 3600,
        "error_rate": metrics["errors"] / max(metrics["total_requests"], 1)
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Sauvegarde du fichier
api_file_path = '/tmp/api.py'
with open(api_file_path, 'w') as f:
    f.write(api_code)

print(f"Fichier API créé : {api_file_path}")
print(f"\nPour démarrer l'API :")
print(f"  cd /tmp && uvicorn api:app --host 0.0.0.0 --port 8000 --reload")

## 4. Dockerfile pour Conteneurisation

In [ ]:
# Création du Dockerfile
dockerfile_content = '''
# Dockerfile pour API Housing Price Prediction
FROM python:3.11-slim

# Métadonnées
LABEL maintainer="your-email@example.com"
LABEL description="Housing Price Prediction API"

# Variables d'environnement
ENV PYTHONUNBUFFERED=1 \\
    PYTHONDONTWRITEBYTECODE=1 \\
    MODEL_PATH=/app/models/housing_api_model.joblib

# Répertoire de travail
WORKDIR /app

# Installation des dépendances
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copie du code et du modèle
COPY api.py .
COPY models/ ./models/

# Exposition du port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

# Commande de démarrage
CMD ["uvicorn", "api:app", "--host", "0.0.0.0", "--port", "8000"]
'''

dockerfile_path = '/tmp/Dockerfile'
with open(dockerfile_path, 'w') as f:
    f.write(dockerfile_content)

print(f"Dockerfile créé : {dockerfile_path}")

# Création du requirements.txt pour l'API
requirements_content = '''fastapi==0.109.0
uvicorn[standard]==0.27.0
pydantic==2.5.3
numpy==1.26.3
scikit-learn==1.4.0
joblib==1.3.2
'''

requirements_path = '/tmp/requirements_api.txt'
with open(requirements_path, 'w') as f:
    f.write(requirements_content)

print(f"Requirements créé : {requirements_path}")

print(f"\nCommandes Docker :")
print(f"  # Build")
print(f"  docker build -t housing-api:1.0 /tmp")
print(f"  # Run")
print(f"  docker run -d -p 8000:8000 --name housing-api housing-api:1.0")
print(f"  # Logs")
print(f"  docker logs -f housing-api")

## 5. Démarrage de l'API (Background)

Note : Dans un environnement de production, l'API tournerait sur un serveur dédié. Ici, nous la démarrons pour tests.

In [ ]:
# Import pour démarrage de l'API
import subprocess
import time

print("Démarrage de l'API FastAPI en arrière-plan...")
print("Note : L'API sera accessible sur http://localhost:8000")
print("Documentation Swagger : http://localhost:8000/docs")
print("\nPour démarrer manuellement :")
print("  cd /tmp && uvicorn api:app --host 0.0.0.0 --port 8000 --reload")

## 6. Tests de l'API avec Requests

Nous supposons que l'API est démarrée (manuellement ou via Docker).

In [ ]:
import requests

# URL de l'API (adapter selon votre configuration)
API_URL = "http://localhost:8000"

print("=" * 60)
print("TESTS DE L'API")
print("=" * 60)
print(f"\nURL de l'API : {API_URL}")
print("\nNote : Si l'API n'est pas démarrée, ces tests échoueront.")
print("Pour démarrer l'API : cd /tmp && uvicorn api:app --port 8000\n")

In [ ]:
# Test 1 : Root endpoint
print("\n" + "=" * 60)
print("TEST 1 : Root Endpoint (GET /)")
print("=" * 60)

try:
    response = requests.get(f"{API_URL}/")
    print(f"Status Code: {response.status_code}")
    print(f"Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Erreur : {e}")
    print("L'API n'est probablement pas démarrée.")

In [ ]:
# Test 2 : Health check
print("\n" + "=" * 60)
print("TEST 2 : Health Check (GET /health)")
print("=" * 60)

try:
    response = requests.get(f"{API_URL}/health")
    print(f"Status Code: {response.status_code}")
    print(f"Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Erreur : {e}")

In [ ]:
# Test 3 : Prédiction unique
print("\n" + "=" * 60)
print("TEST 3 : Prédiction Unique (POST /predict)")
print("=" * 60)

# Données de test
test_input = {
    "MedInc": 3.5,
    "HouseAge": 20.0,
    "AveRooms": 5.0,
    "AveBedrms": 1.5,
    "Population": 1200.0,
    "AveOccup": 3.0,
    "Latitude": 37.5,
    "Longitude": -122.0
}

print(f"Input:")
print(json.dumps(test_input, indent=2))

try:
    response = requests.post(f"{API_URL}/predict", json=test_input)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Erreur : {e}")

In [ ]:
# Test 4 : Prédictions batch
print("\n" + "=" * 60)
print("TEST 4 : Prédictions Batch (POST /predict/batch)")
print("=" * 60)

# Batch de 3 maisons
batch_input = {
    "data": [
        {  # Maison 1 : prix élevé
            "MedInc": 8.0,
            "HouseAge": 10.0,
            "AveRooms": 7.0,
            "AveBedrms": 2.0,
            "Population": 500.0,
            "AveOccup": 2.5,
            "Latitude": 37.8,
            "Longitude": -122.4
        },
        {  # Maison 2 : prix moyen
            "MedInc": 3.5,
            "HouseAge": 20.0,
            "AveRooms": 5.0,
            "AveBedrms": 1.5,
            "Population": 1200.0,
            "AveOccup": 3.0,
            "Latitude": 37.5,
            "Longitude": -122.0
        },
        {  # Maison 3 : prix bas
            "MedInc": 1.5,
            "HouseAge": 40.0,
            "AveRooms": 3.0,
            "AveBedrms": 1.0,
            "Population": 3000.0,
            "AveOccup": 5.0,
            "Latitude": 34.0,
            "Longitude": -118.0
        }
    ]
}

print(f"Nombre de maisons : {len(batch_input['data'])}")

try:
    response = requests.post(f"{API_URL}/predict/batch", json=batch_input)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Response:")
    result = response.json()
    print(f"Nombre de prédictions : {result['count']}")
    print(f"\nPrédictions :")
    for i, pred in enumerate(result['predictions'], 1):
        print(f"  Maison {i}: {pred['prediction_usd']} (raw: {pred['prediction']:.4f})")
except Exception as e:
    print(f"Erreur : {e}")

In [ ]:
# Test 5 : Validation des inputs (erreur attendue)
print("\n" + "=" * 60)
print("TEST 5 : Validation des Inputs (Erreur Attendue)")
print("=" * 60)

# Input invalide : MedInc négatif
invalid_input = {
    "MedInc": -1.0,  # INVALIDE
    "HouseAge": 20.0,
    "AveRooms": 5.0,
    "AveBedrms": 1.5,
    "Population": 1200.0,
    "AveOccup": 3.0,
    "Latitude": 37.5,
    "Longitude": -122.0
}

print(f"Input (invalide - MedInc négatif):")
print(json.dumps(invalid_input, indent=2))

try:
    response = requests.post(f"{API_URL}/predict", json=invalid_input)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Erreur : {e}")

In [ ]:
# Test 6 : Métriques
print("\n" + "=" * 60)
print("TEST 6 : Métriques (GET /metrics)")
print("=" * 60)

try:
    response = requests.get(f"{API_URL}/metrics")
    print(f"Status Code: {response.status_code}")
    print(f"Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Erreur : {e}")

## 7. Script de Load Testing (Optionnel)

In [ ]:
# Load testing simple
import time
import random

print("=" * 60)
print("LOAD TESTING (50 requêtes)")
print("=" * 60)

n_requests = 50
response_times = []
success_count = 0

print(f"\nEnvoi de {n_requests} requêtes...")

for i in range(n_requests):
    # Génération d'inputs aléatoires
    test_input = {
        "MedInc": random.uniform(0.5, 10.0),
        "HouseAge": random.uniform(1, 50),
        "AveRooms": random.uniform(2, 10),
        "AveBedrms": random.uniform(0.5, 3),
        "Population": random.uniform(100, 5000),
        "AveOccup": random.uniform(1, 8),
        "Latitude": random.uniform(32, 42),
        "Longitude": random.uniform(-125, -114)
    }
    
    try:
        start_time = time.time()
        response = requests.post(f"{API_URL}/predict", json=test_input, timeout=5)
        elapsed = time.time() - start_time
        
        if response.status_code == 200:
            success_count += 1
            response_times.append(elapsed)
        
        if (i + 1) % 10 == 0:
            print(f"  Progression: {i + 1}/{n_requests}")
    
    except Exception as e:
        print(f"  Erreur requête {i + 1}: {e}")

# Statistiques
if response_times:
    print(f"\n" + "=" * 60)
    print("RÉSULTATS LOAD TESTING")
    print("=" * 60)
    print(f"Requêtes envoyées: {n_requests}")
    print(f"Succès: {success_count} ({success_count / n_requests * 100:.1f}%)")
    print(f"Échecs: {n_requests - success_count}")
    print(f"\nTemps de réponse (ms):")
    print(f"  Min:    {min(response_times) * 1000:.2f}")
    print(f"  Max:    {max(response_times) * 1000:.2f}")
    print(f"  Moyenne: {np.mean(response_times) * 1000:.2f}")
    print(f"  Médiane: {np.median(response_times) * 1000:.2f}")
    print(f"  P95:     {np.percentile(response_times, 95) * 1000:.2f}")
    print(f"  P99:     {np.percentile(response_times, 99) * 1000:.2f}")
    
    # Visualisation
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist([t * 1000 for t in response_times], bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Temps de réponse (ms)')
    plt.ylabel('Fréquence')
    plt.title('Distribution des Temps de Réponse')
    plt.axvline(np.mean(response_times) * 1000, color='red', linestyle='--', label='Moyenne')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot([t * 1000 for t in response_times], marker='o', linestyle='-', markersize=3, alpha=0.7)
    plt.xlabel('Requête #')
    plt.ylabel('Temps de réponse (ms)')
    plt.title('Temps de Réponse par Requête')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Aucune requête réussie. L'API n'est probablement pas démarrée.")

## 8. Conclusion

### Points Clés du Déploiement ML

1. **API REST avec FastAPI** :
   - Endpoints clairs : `/predict`, `/predict/batch`, `/health`, `/metrics`
   - Documentation automatique (Swagger UI)
   - Validation des inputs avec Pydantic

2. **Validation et Robustesse** :
   - Pydantic pour valider les inputs (types, ranges, règles custom)
   - Gestion des erreurs avec HTTPException
   - Middleware pour tracking des métriques

3. **Monitoring et Observabilité** :
   - Health checks pour Kubernetes/Docker
   - Métriques : nombre de prédictions, requêtes, erreurs, uptime
   - Headers custom (X-Process-Time)

4. **Conteneurisation avec Docker** :
   - Image légère (python:3.11-slim)
   - Health check intégré
   - Variables d'environnement

### Best Practices Production

- **Versioning** : Versioner les modèles et l'API
- **Logging** : Utiliser des loggers structurés (JSON)
- **Sécurité** : Ajouter authentification (OAuth2, JWT)
- **Rate Limiting** : Limiter le nombre de requêtes par utilisateur
- **Monitoring** : Intégrer Prometheus, Grafana, ELK stack
- **A/B Testing** : Tester plusieurs versions du modèle
- **CI/CD** : Automatiser les tests et déploiements

### Fichiers Générés

- `/tmp/api.py` : Code de l'API FastAPI
- `/tmp/Dockerfile` : Dockerfile pour conteneurisation
- `/tmp/requirements_api.txt` : Dépendances Python

### Commandes Utiles

```bash
# Démarrer l'API localement
uvicorn api:app --host 0.0.0.0 --port 8000 --reload

# Build Docker image
docker build -t housing-api:1.0 .

# Run Docker container
docker run -d -p 8000:8000 --name housing-api housing-api:1.0

# Tester l'API
curl http://localhost:8000/health
```